# Messy medical data: Pandas text processing

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once these date patterns from the text are extracted, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules, the correct date in each note will be recognized and returned as a pandas Series in chronological order.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

The function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
import datetime as dt
import re

def date_sorter():
    pd.set_option('display.max_rows', 500)
    pd.set_option('display.max_colwidth', 1000)
    #------------------------------------------------------mm/dd/yy, mm/dd/yyyy, mm-dd-yy, m/d/yy 
    #mix of 2 digit and 4 digit years
    df1= df.str.findall(r'(\d{1,2}[\/\-]\d{1,2}[\/\-]\d{2,4})\D') 
    # only mm/dd/yy format
    #gather only 2 digit years (not as a list) thats why str[0] is there to avoide returning a list for each item
    df1_1= df.str.findall(r'(\d{1,2}[\/\-]\d{1,2}[\/\-]\d{2})\D').str[0] 
    #change - - to / / in yy format
    df1_2 = df1_1.str.replace(r'\-', r'/')
    df1_3 = pd.to_datetime(df1_2, format='%m/%d/%y')
    #only mm/dd/yyyy format, there are no mm-dd-yyyy items here
    df1_4= df.str.findall(r'(\d{1,2}[\/\-]\d{1,2}[\/\-]\d{4})\D').str[0]
    df1_5 = pd.to_datetime(df1_4, format='%m/%d/%Y')
    df1_f = df1_3.fillna(df1_5)
    #-------------------------------------------------------------------mon dd, yyyy
    df2_1_1= df.str.findall(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) \d{1,2}, \d{2,4}').str[0]
    df2_1 = pd.to_datetime(df2_1_1, format='%b %d, %Y')
    #mon dd yyyy
    df2_2_1= df.str.findall(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) \d{1,2} \d{2,4}').str[0]
    df2_2 = pd.to_datetime(df2_2_1, format='%b %d %Y')
    #mon-dd-yyyy
    df2_3_1= df.str.findall(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\. \d{1,2}, \d{2,4}').str[0]
    df2_3 = pd.to_datetime(df2_3_1, format='%b. %d, %Y')
    #month dd, yyyy
    df2_4_1= df.str.findall(r'(?:January|February|March|April|May|June|July|August|September|October|November|December) \d{1,2}, \d{4}').str[0]
    df2_4 = pd.to_datetime(df2_4_1, format='%B %d, %Y')
    #month dd yyyy
    df2_5_1= df.str.findall(r'(?:January|February|March|April|May|June|July|August|September|October|November|December) \d{1,2} \d{4}').str[0]
    df2_5 = pd.to_datetime(df2_5_1, format='%B %d %Y')
    #month. dd, yyyy
    df2_6_1= df.str.findall(r'(?:January|February|March|April|May|June|July|August|September|October|November|December). \d{1,2}, \d{4}').str[0]
    df2_6 = pd.to_datetime(df2_6_1, format='%B. %d, %Y')
    # joining group 2
    df2_f1 = df2_1.fillna(df2_2)
    df2_f2 = df2_3.fillna(df2_4)
    df2_f3 = df2_5.fillna(df2_6)
    df2_f4 = df2_f1.fillna(df2_f2)
    df2_f = df2_f3.fillna(df2_f4)
    #-------------------------------------------------------------------dd mon yyyy
    df3_1_1= df.str.findall(r'\d{1,2} (?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) \d{4}').str[0]
    df3_1 = pd.to_datetime(df3_1_1, format='%d %b %Y')
    #dd mon yyyy
    df3_2_1= df.str.findall(r'\d{1,2} (?:January|February|March|April|May|June|July|August|September|October|November|December) \d{4}').str[0]
    df3_2 = pd.to_datetime(df3_2_1, format='%d %B %Y')
    # joining group 3
    df3_f = df3_1.fillna(df3_2)
    #-------------------------------------------------------------------mon yyyy
    df5_1_1= df.str.findall(r'(?:\D+ |^|\S)((?:Jan|Feb|Mar|Apr|Jun|Jul|Aug|Sep|Oct|Nov|Dec) \d{4})').str[0]
    df5_1 = pd.to_datetime(df5_1_1, format='%b %Y')
    #month yyyy
    df5_2_1= df.str.findall(r'(?:\D+ |^|\S)((?:Jan|Feb|Mar|Apr|Jun|Jul|Aug|Sep|Oct|Nov|Dec), \d{4})').str[0]
    df5_2 = pd.to_datetime(df5_2_1, format='%b, %Y')
    #month, yyyy
    df5_3_1= df.str.findall(r'(?:\D+ |^|\S)((?:January|February|March|April|May|June|July|August|September|October|November|December) \d{4})').str[0]
    df5_3 = pd.to_datetime(df5_3_1, format='%B %Y')
    #Xmonth yyyyX
    df5_4_1= df.str.findall(r'(?:\D+ |^|\S)((?:January|February|March|April|May|June|July|August|September|October|November|December), \d{4})').str[0]
    df5_4 = pd.to_datetime(df5_4_1, format='%B, %Y')
    # There is no group 4!
    # joining group 5
    df5_f1 = df5_1.fillna(df5_2)
    df5_f2 = df5_3.fillna(df5_4)
    df5_f = df5_f1.fillna(df5_f2)
    #-------------------------------------------------------------------  mm/yyyy 
    df6_1_1= df.str.findall(r'(?:^|[^\/])([01]\d\/\d{4})').str[0]
    df6_1 = pd.to_datetime(df6_1_1, format='%m/%Y')
    #  m/yyyy
    df6_2_1= df.str.findall(r'(?:^|[^\/\d])(\d\/\d{4})').str[0]
    df6_2 = pd.to_datetime(df6_2_1, format='%m/%Y')
    df6_f = df6_1.fillna(df6_2)
    #---------------------------------------------------------------------yyyy
    df7_1_1= df.str.findall(r'(?<!Jan )(?<!Feb )(?<!Mar )(?<!Apr )(?<!May )(?<!Jun )(?<!Jul )(?<!Aug )(?<!Sep )(?<!Oct )(?<!Nov )(?<!Dec )(?<!January )(?<!Janaury )(?<!February )(?<!March )(?<!April )(?<!May )(?<!June )(?<!July )(?<!August )(?<!September )(?<!October )(?<!November )(?<!December )(?<!January, )(?<!February, )(?<!March, )(?<!April, )(?<!Apr, )(?<!May, )(?<!June, )(?<!July, )(?<!August, )(?<!September, )(?<!October, )(?<!November, )(?<!December, )(?<!Decemeber )(?<! \d{2} )(?<! \d{2}, )(?<!\/)((?:19|20)\d{2})').str[0]
    df7_f = pd.to_datetime(df7_1_1, format='%Y')
    #-----------------------------------------------------------missing data 298,313,461,465
    df8_1_1= df.str.findall(r'(?<!Jan )(?<!Feb )(?<!Mar )(?<!Apr )(?<!May )(?<!Jun )(?<!Jul )(?<!Aug )(?<!Sep )(?<!Oct )(?<!Nov )(?<!Dec )(?<!January )(?<!Janaury )(?<!February )(?<!March )(?<!April )(?<!May )(?<!June )(?<!July )(?<!August )(?<!September )(?<!October )(?<!November )(?<!December )(?<!January, )(?<!February, )(?<!March, )(?<!April, )(?<!Apr, )(?<!May, )(?<!June, )(?<!July, )(?<!August, )(?<!September, )(?<!October, )(?<!November, )(?<!December, )(?<!Decemeber )(?<! \d{2} )(?: \d{2}, )(?<!\/)((?:19|20)\d{2})').str[0]
    df8_f = pd.to_datetime(df8_1_1, format='%Y')
    #------------------------------------------------------------------ final join
    df1_2_f = df1_f.fillna(df2_f)
    df1_3_f = df1_2_f.fillna(df3_f)
    df1_5_f = df1_3_f.fillna(df5_f)
    df1_6_f = df1_5_f.fillna(df6_f)
    df1_7_f = df1_6_f.fillna(df7_f)
    df_f= df1_7_f.fillna(df8_f)
          
    dff = pd.DataFrame(df_f)
    dff1= dff.sort_values(by=0)
    dff2= dff1.reset_index()
    result = dff2['index']
    
    return result # Your answer here

date_sorter()